## Tacotron 2 inference code 
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

#### Import libraries and setup matplotlib

In [1]:
import re

from scipy.io.wavfile import write

from libs.audio.audio_processing import griffin_lim
from libs.model.layers import TacotronSTFT
%matplotlib inline
import os
import sys

import IPython.display as ipd
import matplotlib.pylab as plt
sys.path.append('nvidia_waveglow/')
import numpy as np
import torch

from hparams import create_hparams
from train import load_model
from libs.text import text_to_sequence


Bad key text.latex.unicode in file /home/arnas/anaconda3/envs/nvidia-tacotron/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /home/arnas/anaconda3/envs/nvidia-tacotron/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key nbagg.transparent in file /home/arnas/anaconda3/envs/nvidia-tacotron/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 426 ('nbagg.transparent: True')
Yo

In [2]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='lower',
                       interpolation='none')

#### Setup hparams

In [3]:
hparams = create_hparams()
hparams.sampling_rate = 22050

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



#### Count synthesis output files

In [4]:
idx = sum([1 for file in os.listdir("files/synthesis")])

#### Load model from checkpoint

In [5]:
checkpoint_path = "/media/arnas/SSD Disk/inovoice/experiments/tacotron/nijole-1/checkpoint_47000"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
# _ = model.cuda().eval().half()

<All keys matched successfully>

#### Prepare text input

In [13]:

# text = "Išeĩkite į\u0303 lau\u0303ką, pasùkite į\u0303 kaĩrę ir\u0303 eĩkite tie\u0303siai iki\u0300 pa\u0303stato gãlo."
text = "Sveikì atvỹkę į bìznis steìdijum vèst, kuõ galiù jùms padė̃ti?"
# text = "Víenas liùdytojas teismè apibū̃dino Doilį kaĩp relìginį fanãtiką, sapaliojantį nerišliàs pamìšėliškas nesą́mones."
# text = "Sveiki\u0300, kuo\u0303 galė\u0301čiau ju\u0300ms padė\u0301ti?"
sequence = np.array(text_to_sequence(text, ['basic_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()

#### Decode text input and plot results

In [14]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))

#### Load TacotronSTFT and convert mel-spectrogram to spectrogram

In [15]:
taco_stft = TacotronSTFT(
    hparams.filter_length, hparams.hop_length, hparams.win_length,
    sampling_rate=hparams.sampling_rate)
mel_decompress = taco_stft.spectral_de_normalize(mel_outputs_postnet)
mel_decompress = mel_decompress.transpose(1, 2).data.cpu()
spec_from_mel_scaling = 1000
spec_from_mel = torch.matmul(mel_decompress[0], taco_stft.mel_basis)
spec_from_mel = spec_from_mel.transpose(0, 1).unsqueeze(0)
spec_from_mel = spec_from_mel * spec_from_mel_scaling

In [16]:
waveform = griffin_lim(torch.autograd.Variable(spec_from_mel[:, :, :-1]),
                       taco_stft.stft_fn, 60)
audio_numpy = waveform[0].data.cpu().numpy()

In [17]:
ipd.Audio(audio_numpy, rate=hparams.sampling_rate)

In [18]:
def prepare_output_filepath(output_dir, sentence):
    filename = re.sub(r'[^a-ž ]', '', sentence.lower())
    filename = re.sub(r"[\u0300\u0301\u0303]", '', filename)
    filename = re.sub(r'[ ]', '_', filename)
    return f"{output_dir}/{filename}.wav"
    # return f"{output_dir}/{checkpoint_name}_{filename}.wav"

In [19]:
output_dir = f"files/synthesis"
filepath = prepare_output_filepath(output_dir, text)
print(f"Writing {filepath}")
write(filepath, hparams.sampling_rate, audio_numpy)

Writing files/synthesis/sveiki_atvykę_į_biznis_steidijum_vest_kuo_galiu_jums_padėti.wav
